# Quickstart for clustering Documentation — Full Suite

<!-- This interactive notebook guides you through the process of documenting a model with the ValidMind Developer Framework. It uses the [Bank Customer Churn Prediction](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from Kaggle to train a simple classification model.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train a simple classification model
- Running a ValidMind test suite to quickly generate documention about the data and model -->

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready. When running this notebook locally, this includes installing any missing prerequisite modules that you discover with `pip install`. 

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [1]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

For now, get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to Go to **Documentation Projects** and click **Create new project**.

3. Select **[Demo] Model for Clustering** and **Initial Validation** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
# Replace this placeholder with the code snippet from your own project ##

import validmind as vm

vm.init(
  api_host = "...",
  api_key = "...",
  api_secret = "...",
  project = "...",
) 


/Users/anilsorathiya/Library/Caches/pypoetry/virtualenvs/validmind-1QuffXMV-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2023-10-30 09:25:31,460 - INFO(validmind.api_client): Connected to ValidMind. Project: Newstrack demo - Initial Validation (clobz3bpl00058mrlq4l6zttv)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [3]:
import numpy as np
import pandas as pd
from time import time
import pyarrow.feather as feather

from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

from sklearn.cluster import KMeans


### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [4]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library, along with a second, different dataset you can try as well. 

To be able to use either sample dataset, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [5]:
read_df = feather.read_feather('../20231026 SampleData_filtered.f')

df = read_df.drop(['language_code', 'title_snippet_processed', 'Prob_LogReg_L2_model', 'distance center'],axis=1)
target_column = "Cluster"
cluster_labels = my_dict = dict(zip(range(len(np.unique(df.Cluster))), np.unique(df.Cluster)))


### Initialize a ValidMind dataset object

Before you can run a test suite, which are just a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module. 

This function takes a number of arguments: 

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset 
- `cluster_labels` — the list of class labels used for classification model training

In [9]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=target_column,
    class_labels=cluster_labels
)

2023-10-30 09:25:51,214 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests. 

### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Scale data using `StandardScaler`. 
- Initialize KMeans clustering algorithm: Creates an `KMeans` object with `k-means++` method.
- Fit the model: Trains the model on `x_train` and `y_train`.

In [10]:
 from validmind.datasets.cluster import digits as demo_dataset
train_df, validation_df, test_df = demo_dataset.preprocess(df)

x_train = train_df.drop(target_column, axis=1)
y_train = train_df[target_column]
x_val = validation_df.drop(target_column, axis=1)
y_val = validation_df[target_column]
x_test = test_df.drop(target_column, axis=1)
y_test = test_df[target_column]
x_train = pd.concat([x_train, x_val], axis=0)
y_train = pd.concat([y_train, y_val], axis=0)

scale = False
if scale:
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.fit_transform(x_val)
    x_test = scaler.fit_transform(x_test)


n_clusters = len(list(cluster_labels.values()))
model = KMeans(init="k-means++", n_clusters=n_clusters, n_init=4, random_state=0)
model = model.fit(x_train)

### Initialize the training and test datasets

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [11]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    target_column=target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    target_column=target_column
)

2023-10-30 09:25:54,512 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-10-30 09:25:54,715 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Initialize a model object 

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [12]:
vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

/Users/anilsorathiya/Library/Caches/pypoetry/virtualenvs/validmind-1QuffXMV-py3.9/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
/Users/anilsorathiya/Library/Caches/pypoetry/virtualenvs/validmind-1QuffXMV-py3.9/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


### Run the full suite of tests

This is where it all comes together: you are now ready to run the full suite of tests for the model as defined by the documentation template you looked at earlier. 

The [`vm.run_documentation_tests`](https://docs.validmind.ai/validmind/validmind.html#run_documentation_tests) function finds and runs every test specified in the template and then uploads all the documentation and test artifacts that get generated to the ValidMind AI Risk Platform. 

The function takes two arguments:

- `dataset`: The data to be tested, specified as `vm_dataset`.
- `model`: The model to be used for testing, specified as `vm_model`.

The variable `full_suite` then holds the result of these tests.

In [13]:
config = {
    "validmind.model_validation.sklearn.HyperParametersTuning":{
        "param_grid":{"n_clusters":range(40, 60)}
    },
    "validmind.model_validation.sklearn.KMeansClustersOptimization":{
        "n_clusters":range(40, 60)
    }
}
vm.run_documentation_tests(model=vm_model,section="model_development_clustering", config=config)

NewstrackApplicationTestSuite(sections=[TestSuiteSection(tests=[], section_id='__top_level__', description=None), TestSuiteSection(tests=[TestSuiteTest(test_id='validmind.model_validation.ModelMetadata', _test_class=<class 'validmind.tests.model_validation.ModelMetadata.ModelMetadata'>, _test_instance=ModelMetadata(test_context=TestContext(dataset=None, model=<validmind.models.sklearn.SKlearnModel object at 0x2bc8847c0>, models=None, context_data=None), result=TestSuiteMetricResult(result_id="validmind.model_validation.ModelMetadata", metric, figures), params={'validmind.model_validation.sklearn.LocalOutlierFactorPlot': {'n_neighbors': 7, 'contamination': 0.01}}), result=TestSuiteMetricResult(result_id="validmind.model_validation.ModelMetadata", metric, figures)), TestSuiteTest(test_id='validmind.data_validation.DatasetSplit', _test_class=<class 'validmind.tests.data_validation.DatasetSplit.DatasetSplit'>, _test_instance=DatasetSplit(test_context=TestContext(dataset=None, model=<validm

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI: 

1. Log back into the [Platform UI](https://app.prod.validmind.ai) 

2. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Documentation**.

3. Expand the following sections and take a look around:
   
   - **2. Data Preparation**
   - **3. Model Development**

What you can see now is a much more easily consumable version of the documentation, including the results of the tests you just performed, along with other parts of your documentation project that still need to be completed. There is a wealth of information that gets uploaded when you run the full test suite, so take a closer look around, especially at test results that might need attention (hint: some of the tests in 2.1 Data description look like they need some attention).

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).

